In [4]:
import os
from io import StringIO

import pandas as pd
import requests
from dotenv import load_dotenv

In [5]:
load_dotenv()
url = os.getenv('MBAJK_URL')
response = requests.get(url)

df = pd.read_json(StringIO(response.text))
df.drop(columns=['contract_name','number'], inplace=True, axis=1)
df['last_update'] = pd.to_datetime(df['last_update'], unit='ms')

df_position = pd.json_normalize(df['position'])
df = pd.concat([df, df_position], axis=1)
df.drop(columns=['position'], inplace=True)

In [6]:
for name, group in df.groupby('name'):
    filename = f'../data/raw/mbajk/{name}.csv'
    if os.path.exists(filename):
        existing_df = pd.read_csv(filename, parse_dates=['last_update'])
        if existing_df['last_update'].max() < group['last_update'].max():
            group.to_csv(filename, mode='a', header=False, index=False)
    else:
        group.to_csv(filename, index=False)